In [24]:
from time import perf_counter
from __future__ import annotations

class Stopwatch:
    '''This is the implementation of a stopwatch, can use it to perform basic actions as start, stop, reset. Has a decorator
    implementation to use it on custom functions and has a context manager implementation too.'''
    def __init__(self):
      self._t_start = None
      self._t_stop = None
      self._running = False
      self._elapsed = 0.0
      self._lap_counter = 0
      self.laps = {}  
    
    # ---------- basic methods -----------
    def __repr__(self):
        return f"Stopwatch(elapsed={(self.ms):.6f}ms, running={self._running})"

    def start(self):
        '''starts the stop watch, changes the internal state to running'''
        if not self._running:
            self._t_start = perf_counter()
            self._running = True

    def stop(self):
        '''stops the stop watch (checks if a start was called or not, else raises an error), 
           re-calibrates the internal state'''
        if not self._running: 
            raise RuntimeError('Cannot stop a watch without starting it first! DUH!')
        self._t_stop = perf_counter()
        self._elapsed += self._t_stop - self._t_start
        self._t_start = None
        self._running = False

    def reset(self):
        '''resets the class attributes to initial values'''
        self._t_start, self._t_stop, self._elapsed = None, None, None
        self._running = False

    def lap(self):
        '''store laps for time tracking efficiently'''
        if not self._running:
            raise RuntimeError('Cannot lap without starting the clock first! DUH!')
        time = perf_counter() - self._t_start
        self.laps[self._lap_counter] = time
        self._lap_counter += 1

    # ---------- dynamic class attributes -----------
    @property
    def seconds(self):
        '''class attribute that stores time elapsed in seconds'''
        if self._running:
            return self._elapsed + (perf_counter() - self._t_start)
        return self._elapsed 

    @property
    def ms(self):
        '''class attribute that stores time elapsed in milliseconds'''
        return 1000 * self.seconds

    # ---------- Decorator -----------
    def decorator(func):
        def wrapper(*args, **kwargs):
            
            sw = Stopwatch()
            sw.start()
            try:
                return func(*args, **kwargs)
            finally:
                sw.stop()
                print(f'Total execution time of {func.__name__}: {sw.seconds}(s)')
        return wrapper

    # ---------- Context Manager implementation -----------
    def __enter__(self):
        self.start()
        return self

    def __exit__(self, exc_type, exc, tb):
        if self._running:
            self.stop()
        

In [16]:
watch = Stopwatch()
watch.start()
x = []
for i in range(100000):
    x.append(i)

watch.stop()
watch.start()
for i in range(100000):
    x.append(i)
watch.stop()
print(watch.ms, watch.seconds)
watch

21.87947200002327 0.02187947200002327


Stopwatch(elapsed=21.879472ms, running=False)

In [17]:
watch.reset()

In [18]:
watch.reset()

In [19]:
@Stopwatch.decorator
def count_100():
    x = list()
    for i in range(100):
        x.append(i)

count_100()

Total execution time of count_100: 8.577000016884995e-06(s)


In [20]:
# testing laps
import time

sw = Stopwatch()
sw.start()
print('hello rohan')
time.sleep(2)
print('first lap')
sw.lap()
time.sleep(2)
print('second_lap')
sw.lap()
time.sleep(2)
sw.stop()
sw

hello rohan
first lap
second_lap


Stopwatch(elapsed=6001.510129ms, running=False)

In [21]:
# testing the laps feature
sw.laps

{0: 2.000647889999982, 1: 4.001141685999983}

In [22]:
# trying the context manager
with Stopwatch() as sw:
    count_100()

print(sw.seconds)

Total execution time of count_100: 1.0078000059365877e-05(s)
0.00023122900006455893
